In [1]:
from sklearn.model_selection import train_test_split

from prepared_data.get_prepared_data import get_prepared_data
import pandas as pd
import numpy as np
import tensorflow as tf


2022-05-04 17:41:36.722027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 17:41:36.722054: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_org = pd.read_csv('data/train.csv')
train,_,_ = get_prepared_data(train_org,map_target=False)

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


percent of object with nan value and orginals:  42.20, 100.000000
remove history columns [9, 10]
percent of object with nan value and orginals:  38.76, 100.000000
remove object with more that  col 50 % empty...
percent of object with nan value and orginals:  38.76, 100.000000
date to datetime type and remove nan date....
percent of object with nan value and orginals:  28.72, 85.915556
filna coach_id and change to is new coach
percent of object with nan value and orginals:  1.20, 85.915556
fill zero nan data
percent of object with nan value and orginals:  0.00, 85.915556


In [15]:
target = train['target']
train_features = train.drop('target',axis=1)
di = {'home': 2, 'draw': 1, 'away': 0}
target = target.replace(di)

X_train, X_test, y_train, y_test = train_test_split(train_features,target, test_size=0.33,random_state=33)

In [16]:

def build_and_compile_model(norm):
    model = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3 )
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model



In [17]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))


In [18]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 198)              397       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 10)                1990      
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 2,420
Trainable params: 2,023
Non-trainable params: 397
_________________________________________________________________


In [19]:
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=10,
)


score = dnn_model.score(X_test,y_test)
print(score)




Epoch 1/10
1597/1597 [==============================] - 2s 1ms/step - loss: 1.0791 - accuracy: 0.4604 - val_loss: 1.0295 - val_accuracy: 0.4846
Epoch 2/10
1597/1597 [==============================] - 2s 1ms/step - loss: 1.0176 - accuracy: 0.4930 - val_loss: 1.0161 - val_accuracy: 0.4935
Epoch 3/10
1597/1597 [==============================] - 2s 976us/step - loss: 1.0104 - accuracy: 0.4976 - val_loss: 1.0104 - val_accuracy: 0.4970
Epoch 4/10
1597/1597 [==============================] - 1s 916us/step - loss: 1.0069 - accuracy: 0.4995 - val_loss: 1.0110 - val_accuracy: 0.4973
Epoch 5/10
1597/1597 [==============================] - 2s 1ms/step - loss: 1.0046 - accuracy: 0.5033 - val_loss: 1.0093 - val_accuracy: 0.4978
Epoch 6/10
1597/1597 [==============================] - 2s 1ms/step - loss: 1.0028 - accuracy: 0.5026 - val_loss: 1.0100 - val_accuracy: 0.4977
Epoch 7/10
1597/1597 [==============================] - 2s 1ms/step - loss: 1.0012 - accuracy: 0.5065 - val_loss: 1.0093 - val_accur

AttributeError: 'Sequential' object has no attribute 'score'

In [20]:
test_loss, test_acc = dnn_model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)


983/983 - 1s - loss: 1.0166 - accuracy: 0.4918 - 517ms/epoch - 526us/step

Test accuracy: 0.4918449819087982
